# PyTorch Third Attempt

In [1]:
# # Check out available CPU and GPU memory
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

def print_CPU_GPU_info(GPUs):
    process = psutil.Process(os.getpid())
    print(f"\nCPU \tRAM Free: {humanize.naturalsize(psutil.virtual_memory().available)}"
          f"    | Proc size: {humanize.naturalsize(process.memory_info().rss)}")
    if GPUs[0]: 
        for i,gpu in enumerate(GPUs):
            print(f"GPU {i} \tRAM Free: {gpu.memoryFree/1000:.3f} GB  "
                  f"| Used: {gpu.memoryUsed/1000:.3f} GB"
                  f"\t| Utilization: {gpu.memoryUtil*100:3.0f}% | "
                  f"Total Memory: {gpu.memoryTotal/1000:.3f} GB")
    else: print(f"Not on a GPU")


  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [2]:
print_CPU_GPU_info(GPU.getGPUs())


CPU 	RAM Free: 12.9 GB    | Proc size: 143.1 MB
GPU 0 	RAM Free: 11.441 GB  | Used: 0.000 GB	| Utilization:   0% | Total Memory: 11.441 GB


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable scrollable cells

<IPython.core.display.Javascript object>

In [4]:
! git clone https://github.com/samryan18/chess-ray-vision
! git clone https://github.com/mukundv7/crvdataset
! mv chess-ray-vision/clean_notebooks/* .
! mkdir train_full
! mv crvdataset/chess-positions/train-full/* train_full/

Cloning into 'chess-ray-vision'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 80 (delta 11), reused 77 (delta 8), pack-reused 0
Unpacking objects: 100% (80/80), done.
Cloning into 'crvdataset'...
remote: Enumerating objects: 39509, done.
remote: Counting objects: 100% (39509/39509), done.
remote: Compressing objects: 100% (39378/39378), done.
remote: Total 39509 (delta 150), reused 39484 (delta 128), pack-reused 0
Receiving objects: 100% (39509/39509), 770.54 MiB | 32.50 MiB/s, done.
Resolving deltas: 100% (150/150), done.
Checking out files: 100% (39691/39691), done.


# Setup Stuff

In [5]:
# Pytorch Colab Setup
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
  
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

    100% |████████████████████████████████| 614.8MB 27kB/s 
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
cuda:0


In [6]:
## Required packages (Install in Colab)
!pip install tensorflow
!pip install scipy
!pip install numpy
!pip install Pillow
!pip install image

In [7]:
import torchvision
import torch 
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np;
from torch.utils.data import Dataset, DataLoader
import time, datetime
from pytorch_general.pytorch_helper import imshow
from pytorch_general.tensorboard_helper import Logger
from tqdm import tqdm_notebook

from random import randint


from PIL import Image
from pathlib import Path

from typing import Callable
import torch
import dill
import torch.optim as optim
device =torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
import numpy as np
import re
piece_symbols = 'prbnkqPRBNKQ'

def onehot_from_fen(fen):
    eye = np.eye(13)
    output = np.empty((0, 13))
    fen = re.sub('[-]', '', fen)

    for char in fen:
        if(char in '12345678'):
            output = np.append(
              output, np.tile(eye[12], (int(char), 1)), axis=0)
        else:
            idx = piece_symbols.index(char)
            output = np.append(output, eye[idx].reshape((1, 13)), axis=0)

    return output

def fen_from_onehot(one_hot):
    output = ''
    for j in range(8):
        for i in range(8):
            if(one_hot[j][i] == 12):
                output += ' '
            else:
                output += piece_symbols[one_hot[j][i]]
        if(j != 7):
            output += '-'

    for i in range(8, 0, -1):
        output = output.replace(' ' * i, str(i))

    return output

class_prob = onehot_from_fen('4kN1N-B1P5-QQ3B2-R1n1b3-8-1p2P3-1K6-6b1')

# one_hot = np.zeros((64, 13))
# one_hot[np.arange(64,13), class_labels] = 1
# # class_labels
# np.shape(class_prob)

In [0]:
def load_batch(directory='train_full/', batch_size=32):
    '''
    Probably a better way to do this using something like this:
    https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
    
    This loads a single random batch from the training set.
    '''
    pathlist = list(Path(directory).glob('**/*.jpeg'))
    labels = []
    images = []
    n_files = len(pathlist)
    random_indicies = [randint(0, n_files) for _ in range(batch_size)]

    for path in [pathlist[x] for x in random_indicies]:
        label = str(path).split(directory)[1].split(f'.')[0]
        label = onehot_from_fen(label)

        img = np.asarray(Image.open(str(path))).astype('uint8')
        labels.append(label)
        images.append(img)
        
    test_images, test_labels = (images, labels) # TODO
        
    return images, labels, test_images, test_labels




class CustomChessDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, transform=None, root='train_full/', train=True):
#         train_images, train_labels, test_images, test_labels = load_datasets()
        self.transform = transform

        self._train = train
            
        self.root = root
        self.pathlist = list(Path(self.root).glob('**/*.jpeg'))
        self.n_files = len(self.pathlist)

        self.to_tensor = transforms.ToTensor()


    def __len__(self):
        return self.n_files

    def __getitem__(self, idx):
        path = self.pathlist[idx]
        label = str(path).split(self.root)[1].split(f'.')[0]
        label = onehot_from_fen(label)
        img = np.asarray(Image.open(str(path))).astype('uint8')

        img_as_img = Image.fromarray(img)
        img_as_img = img_as_img.convert('L')
        img_as_img = self.transform(img_as_img)
        
        
#         _,class_labels = torch.max(self.to_tensor(self.labels[idx]).long().to(device),1) 

        return (self.to_tensor(img_as_img), 
                label)


In [0]:
# NEW FROM KERAS VERSION

import glob
from random import shuffle
from skimage.util.shape import view_as_blocks
from skimage import transform as sktransform
from skimage import io

piece_symbols = 'prbnkqPRBNKQ'

def fen_from_filename(filename):
    base = os.path.basename(filename)
    return os.path.splitext(base)[0]

def process_image(img, downsample_size = 200):
    square_size = int(downsample_size/8)
    img_read = io.imread(img)
    img_read = sktransform.resize(
      img_read, (downsample_size, downsample_size), mode='constant')
    tiles = view_as_blocks(img_read, block_shape=(square_size, square_size, 3)).squeeze(axis=2)
    return tiles.reshape(64, square_size, square_size, 3)    

class CustomChessDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, 
                 transform=None,
                 root='train_full',
                 train_size = 10000,
                 test_size = 3000,
                 downsample_size = 200,
                 train=True):

        self._train = train
        self.downsample_size = downsample_size
            
        self.root = root
        self.pathlist = list(Path(self.root).glob('**/*.jpeg'))
        self.n_files = len(self.pathlist)
        
        self.train_size = train_size
        self.test_size = test_size

        self.train = glob.glob(f"{root}/*.jpeg")
        self.test = glob.glob(f"{root}/*.jpeg")

        shuffle(self.train)
        shuffle(self.test)

        self.train = self.train[:self.train_size]
        self.test = self.test[:self.test_size]


    def __len__(self):
        return self.train_size

    def __getitem__(self, idx):
        img = self.train[idx]
        label = onehot_from_fen(fen_from_filename(img))
        img_as_img = process_image(img, downsample_size = self.downsample_size)

        return ((torch.from_numpy(img_as_img).float()), 
                label)


In [0]:
# Define a transform to normalize the data
batch_size=10 # this needs to be small ish bc bigger models will scale memory usage exponentially
downsample_size=200
transform = transforms.Compose([transforms.Resize(downsample_size)])

train_dataset = CustomChessDataset(root='train_full/', train=True, downsample_size=downsample_size)
test_dataset = CustomChessDataset(root='train_full/', train=False, downsample_size=downsample_size)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [0]:
def train_model(model: nn.Module, 
                log_dir: str,
                train_loader,
                criterion,
                optimizer,
                num_epochs,
               log_freq,
               max_per_epoch=-1) -> nn.Module:
    t = datetime.datetime.now()
    now = time.mktime(t.timetuple()) - 1550000000
    logger = Logger(f'{log_dir} ({now})/')
    
    print(now)

    model = model.to(device)
    model.train()

    total_step = len(train_loader)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}')
        running_loss = 0
        step = 0
#         for step, (images, labels) in tqdm_notebook(enumerate(train_loader), total=len(train_loader), unit="mini-batches"):
        for step, (images, labels) in enumerate(train_loader):

            images, labels = images.to(device), labels.long().to(device)


            output = model(images).to(device)
            _,class_labels = torch.max(labels,1) 
            loss = criterion(output, class_labels).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute accuracy
            _, argmax = torch.max(output, 1)
            accuracy = float((class_labels == argmax.squeeze()).float().mean().cpu())

            running_loss += float(loss.item())
            
            del(images)
            del(labels)
            del(class_labels)
            
            if step % log_freq == 0:

                overall_step = epoch*total_step + step

                # 1. Log scalar values (scalar summary)
                info = { 'loss': loss.item(), 'accuracy': accuracy }

                for key, value in info.items():
                    logger.scalar_summary(key, value, overall_step)

                # 2. Log values and gradients of the parameters (histogram summary)
                for key, value in model.named_parameters():
                    key = key.replace('.', '/')
                    logger.histo_summary(key, value.data.cpu().numpy(), overall_step)
                    try:
                        logger.histo_summary(key+'/grad', value.grad.data.cpu().numpy(), overall_step)
                    except (AttributeError):
                        # During transfer learning some of the variables don't have grads
                        pass
            
            if max_per_epoch > 0 and step > max_per_epoch:
                break
        
        print(f"{epoch}: Training loss: {running_loss/len(train_loader)}")
        print(f"{epoch}: Training accuracy: {accuracy}")
 
    return model


def test_model(model, criterion, test_loader) -> float:
    model = model.to(device)
    correct = 0
    total = 0
    accuracies = []
    losses = []
    total_step = len(test_loader)
    with torch.no_grad():
        for i in range(total_step):
            for  images, labels in test_loader:
                images, labels = images.to(device), labels.long().to(device)
                _,class_labels = torch.max(labels,1) 

                output = model(images).to(device)
                loss = criterion(output, class_labels)
                losses.append(float(loss.item()))


                # Compute accuracy
                _, argmax = torch.max(output, 1)
                accuracy = float((class_labels == argmax.squeeze()).float().mean().cpu())
                accuracies.append(accuracy)
                
    print(f'Accuracy of the network on test images: {np.average(accuracies)}')
    print(f'Avg. Loss of the network on test images: {np.average(losses)}')

    return np.average(accuracies)

In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1) 

class SimpleCNN(torch.nn.Module):
    def __init__(self, batch_size):
        super(SimpleCNN, self).__init__()
        self.name = 'SimpleCNN'
        self.batch_size=batch_size
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1),
            nn.LeakyReLU(negative_slope=0.1))
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU())
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU())
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(
            nn.Linear(32*19*19, 256),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.fc2 = nn.Sequential(
            nn.Linear(256, 13))

    def forward(self, x):
        x = x.reshape(self.batch_size*64,3,25,25)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
#         print(f'xsize: {x.size()}')
        x = x.reshape(self.batch_size,64,13)

        return(x)
    
class BiggerCNN(torch.nn.Module):
    def __init__(self, batch_size):
        super(BiggerCNN, self).__init__()
        self.name = 'BiggerCNN'
        self.batch_size=batch_size
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
            nn.LeakyReLU(negative_slope=0.1))
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU())
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU())
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(
            nn.Linear(64*19*19, 512),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.fc3 = nn.Sequential(
            nn.Linear(256, 13))

    def forward(self, x):
        x = x.reshape(self.batch_size*64,3,25,25)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

#         print(f'xsize: {x.size()}')
        x = x.reshape(self.batch_size,64,13)

        return(x)
    
class BatchNormBiggerCNN(torch.nn.Module):
    def __init__(self, batch_size):
        super(BatchNormBiggerCNN, self).__init__()
        self.name = 'BatchNormBiggerCNN'
        self.batch_size=batch_size
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(p = 0.1))
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU())
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(
            nn.Linear(64*19*19, 512),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p = 0.1))
        self.fc3 = nn.Sequential(
            nn.Linear(256, 13))

    def forward(self, x):
        x = x.reshape(self.batch_size*64,3,25,25)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

#         print(f'xsize: {x.size()}')
        x = x.reshape(self.batch_size,64,13)

        return(x)

In [0]:
# from torchsummary import summary

# summary(net, input_size=(64, 32, 25, 25))

In [21]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Ngrok already installed
Tensorboard Link: http://aec27de6.ngrok.io


In [0]:
num_epochs = 3
log_freq=20

# 0.0005 best so far
learning_rates = [ 0.0004, 0.0002, 0.0007]

for learning_rate in learning_rates:
    
    net = BiggerCNN(batch_size=batch_size)

    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    log_dir = f'./logs/{net.name}_lr{learning_rate}'
    criterion = nn.CrossEntropyLoss().to(device)

    model = train_model(net,
                    log_dir,
                    train_loader,
                    criterion,
                    optimizer,
                    num_epochs, 
                    log_freq,
                    max_per_epoch=-1) # max per epoch is a debugging thing

# final_acc = test_model(model, criterion, test_loader)

4252158.0
Epoch 0
0: Training loss: 1.2905190806314348
0: Training accuracy: 0.9769230484962463
Epoch 1


In [0]:
%matplotlib inline

# from matplotlib.pyplot import imshow
# import matplotlib.pyplot as plt
from utils.draw_chess_boards import *

renderer = DrawChessPosition(delimiter='-')
fen = "r2q1rk1/pp2ppbp/1np2np1/2Q3B1/3PP1b1/2N2N2/PP3PPP/3RKB1R"
fen = "rnbqkbnr-pppppppp-8-8-8-8-PPPPPPPP-RNBQKBNR"
board = renderer.draw(fen)
renderer.show(board)